## OSMNX O-D pairs creation

Created 10 random O-D pairs. The origins_gdf and destinations_gdf in Data Folder is created in this notebook.

In [ ]:
import geopandas as gpd
import osmnx as ox
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim

## Create O-D

In [ ]:
# generate G of LA County
G = ox.graph_from_place('Los Angeles County, CA, USA', network_type='drive')

In [ ]:
fig, ax = ox.plot_graph(G)
plt.show()

In [ ]:
# convert to GeoDataFrames
nodes_gdf, edges_gdf = ox.graph_to_gdfs(G)

In [ ]:
edges_gdf

In [ ]:
# generate random points LA County
def sample_points(G, n, seed=None):
    if seed is not None:
        np.random.seed(seed)
    gdf_edges = ox.graph_to_gdfs(G, nodes=False)[['geometry', 'length']]
    weights = gdf_edges['length'] / gdf_edges['length'].sum()
    idx = np.random.choice(gdf_edges.index, size=n, p=weights)
    lines = gdf_edges.loc[idx, 'geometry']
    return lines.interpolate(lines.length * np.random.rand())

In [ ]:
# sample 10 points
origins = sample_points(G, 10, seed=11)
destinations = sample_points(G,10, seed=22)

In [ ]:
origins

In [ ]:
destinations

With Google maps, found the address of the O-D points.

In [ ]:
import googlemaps
from datetime import datetime

In [ ]:
# Jessica Google API Key
key = 'Jessica Google API Key'

In [ ]:
gmaps = googlemaps.Client(key=key)

In [ ]:
# origins_address

origins_addresses = []

for point in origins:
    lat, lon = point.y, point.x
    reverse_geocode_result = gmaps.reverse_geocode((lat, lon))
    if reverse_geocode_result:
        address = reverse_geocode_result[0]['formatted_address']
        origins_addresses.append(address)
    else:
        origins_addresses.append("no_address")

In [ ]:
origins_addresses

In [ ]:
# destinations_address

destinations_addresses = []

for point in destinations:
    lat, lon = point.y, point.x
    reverse_geocode_result = gmaps.reverse_geocode((lat, lon))
    if reverse_geocode_result:
        address = reverse_geocode_result[0]['formatted_address']
        destinations_addresses.append(address)
    else:
        destinations_addresses.append("no_address")

In [ ]:
destinations_addresses

In [ ]:
# convert origins to GeoDataFrame and add 'address' column
origins_gdf = gpd.GeoDataFrame(origins, columns=['geometry'])
origins_gdf['address'] = origins_addresses

In [ ]:
origins_gdf

In [ ]:
# convert destinations to GeoDataFrame and add 'address' column
destinations_gdf = gpd.GeoDataFrame(destinations, columns=['geometry'])
destinations_gdf['address'] = destinations_addresses

In [ ]:
destinations_gdf

In [ ]:
origins_gdf['geometry'] = origins_gdf['geometry'].apply(lambda x: x.wkt)
destinations_gdf['geometry'] = destinations_gdf['geometry'].apply(lambda x: x.wkt)

In [ ]:
# export